##Loading Data

In [1]:
!pip install datasets
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
emotion_dataset = load_dataset("emotion");

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [2]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [3]:
emotion_train = emotion_dataset['train']
print(emotion_train[0])
print(emotion_train.column_names)
print(emotion_train.features)

{'text': 'i didnt feel humiliated', 'label': 0}
['text', 'label']
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}


In [4]:
import pandas as pd
emotion_dataset.set_format(type="pandas")
train = emotion_dataset["train"][:]
test = emotion_dataset["test"][:]
val = emotion_dataset["validation"][:]
print(train)
print(train.shape)

                                                    text  label
0                                i didnt feel humiliated      0
1      i can go from feeling so hopeless to so damned...      0
2       im grabbing a minute to post i feel greedy wrong      3
3      i am ever feeling nostalgic about the fireplac...      2
4                                   i am feeling grouchy      3
...                                                  ...    ...
15995  i just had a very brief time in the beanbag an...      0
15996  i am now turning and i feel pathetic that i am...      0
15997                     i feel strong and good overall      1
15998  i feel like this was such a rude comment and i...      3
15999  i know a lot but i feel so stupid because i ca...      0

[16000 rows x 2 columns]
(16000, 2)


## Using Training Data

**TF-IDF on Training data** - train['text'] and train['labels']

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

print("\nTF-IDF: Prints shape/type data corpus--------------------------------")
corpus = train['text']
corpus = corpus.values # now numpy data type
print("corpus.shape ---> ", corpus.shape)
print("type(corpus) ---> ", type(corpus))

# //Look below...
print("\nTF-IDF: Vecotorize/finish preprocessing data-------------------------")
vectorizer = TfidfVectorizer(analyzer= 'word', lowercase = True) # sets lowercase
text_train = vectorizer.fit(corpus)  # builds the vocabulary
text_train = vectorizer.transform(corpus)
print(text_train[2:3]) #<--prints a compressed view of (0, 14)	  0.5773502691896257

print("\nTF-IDF: Prints final Fit/Transformed corpus--------------------------")
print(text_train.toarray())

print("\ntext_train.shape --->", text_train.shape) #<-- (2, 21) means 2 pieces, 21 unique words in total

# now, handle train['label']
label_train = train["label"]
print(label_train[0:5])
print(type(label_train))


TF-IDF: Prints shape/type data corpus--------------------------------
corpus.shape --->  (16000,)
type(corpus) --->  <class 'numpy.ndarray'>

TF-IDF: Vecotorize/finish preprocessing data-------------------------
  (0, 4924)	0.08574826485322755
  (0, 5731)	0.5409161412176642
  (0, 5792)	0.3978238543532009
  (0, 6553)	0.18280349773875593
  (0, 8442)	0.45655358369069104
  (0, 10140)	0.36281022944443825
  (0, 13622)	0.11711560670949034
  (0, 15037)	0.39317619798459597

TF-IDF: Prints final Fit/Transformed corpus--------------------------
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

text_train.shape ---> (16000, 15186)
0    0
1    0
2    3
3    2
4    3
Name: label, dtype: int64
<class 'pandas.core.series.Series'>


**TF-IDF on Validation data** - val['text'] and val['labels']

In [6]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import numpy as np
from scipy.sparse import csr_matrix

#validation data into tf-idf
print("\nTF-IDF: Prints shape/type data corpus--------------------------------")
text_val = val['text']
text_val = text_val.values # now numpy data type
print("text_val.shape ---> ", text_val.shape)
print("type(text_val) ---> ", type(text_val))

print("\nTF-IDF: Vecotorize/finish preprocessing data-------------------------")
text_val = vectorizer.transform(text_val) #<--prints a compressed view of (,) CHANGE

print("\nTF-IDF: Prints final Fit/Transformed corpus--------------------------")
print(text_val.toarray())
print("text_val.shape --->", text_val.shape) #<-- (2, 21) means 2 pieces, 21 unique words in total

print("\nTF-IDF: Turns labels into numpy data type--------------------------------")
label_val = val['label']
label_val = label_val.values # now numpy data type
print("type(text_val) ---> ", type(text_val))

# # Training Y with LabelEncoder
# print("\nTF-IDF: Prints Fitted/Transformed labels---------------------------------------")
# Valid_Y = train["label"]
# Encoder = LabelEncoder()
# Valid_Y_Tfidf = Encoder.fit_transform(Valid_Y)
# print("Valid_Y_Tfidf --->", Valid_Y_Tfidf)

#Val_X_Tfidf is the input for the validation for svm, and Valid_Y_Tfidf is the predicted output for svm


TF-IDF: Prints shape/type data corpus--------------------------------
text_val.shape --->  (2000,)
type(text_val) --->  <class 'numpy.ndarray'>

TF-IDF: Vecotorize/finish preprocessing data-------------------------

TF-IDF: Prints final Fit/Transformed corpus--------------------------
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
text_val.shape ---> (2000, 15186)

TF-IDF: Turns labels into numpy data type--------------------------------
type(text_val) --->  <class 'scipy.sparse._csr.csr_matrix'>


**SVM**

In [7]:
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# fitting training data
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(text_train, label_train) #inputs, outputs

# predict the labels on validation
predictions_SVM = SVM.predict(text_val)

In [8]:
#printing scores...
# Use accuracy_score function to get the accuracy (from article) ask if needed!!
print("SVM Accuracy Score -> ", accuracy_score(label_val, predictions_SVM)*100)

# Use metrics from sklearn function to get the accuracy (from article)
# metrics.name_score(ground truth, predicted)
print("SVM Confusion Matrix Score >>> \n", metrics.confusion_matrix(label_val, predictions_SVM))
print("SVM Accuracy Score >>> ", metrics.accuracy_score(label_val, predictions_SVM))
print("SVM Precision Score >>> ", metrics.precision_score(label_val, predictions_SVM, average= 'macro'))
print("SVM Recall Matrix Score >>> ", metrics.recall_score(label_val, predictions_SVM, average= 'macro'))
print("SVM F1-Score Matrix Score >>> ", metrics.f1_score(label_val, predictions_SVM, average= 'macro')) # in between p and r, good.

# Can I use this?
# print("\nSVM Classification Report >>> \n", classification_report(label_val, predictions_SVM))

SVM Accuracy Score ->  88.9
SVM Confusion Matrix Score >>> 
 [[512  12   4   8  12   2]
 [ 17 661  14   4   7   1]
 [  3  39 134   1   1   0]
 [ 17  12   1 240   5   0]
 [ 13  10   0  11 174   4]
 [  5   7   0   0  12  57]]
SVM Accuracy Score >>>  0.889
SVM Precision Score >>>  0.8825355792373655
SVM Recall Matrix Score >>>  0.8366373712717831
SVM F1-Score Matrix Score >>>  0.85678796718663
